In [1]:
import pandas as pd
import os
import openai
from dotenv import load_dotenv
import json
import difflib

## Importing new recipes

Once you add a new recipe here, you also need to update the full recipe list below.

In [2]:
#Copy the recipe here
raw_recipe = f"""
Ingredients of Paneer Butter MasalaFor frying Paneer:1 tbsp Oil1 cup Paneer cubes1/2 tsp Salt2 tsp Red chilli powderFor gravy:1/2 tbsp Oil2 cubes Butter1 Black cardamom1 Cinnamon stick5 Green cardamom1 tsp Ginger1 tsp Garlic1/2 cup Onion, chopped1 cup Tomato, chopped7-8 nos Cashews1/4 tsp Green chilli, chopped1/2 tsp Oil2 cubes Butter2 tsp Salt1 tsp Red chilli powder1/2 tsp Coriander powder1/2 tsp Cumin powder1/4 tsp Garam masala1/4 tsp Black pepper1/2 tsp Kasoori methi1/2 tsp Honey3/4 cup Milk1 tbsp Cream1/2 tsp Cardamom powder
"""


In [3]:
#Intitializing variables
load_dotenv() 
ENV_PATH = "./.env" 
openai.api_key = os.getenv('api_key')


In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [5]:
def recipe_json(text):
    prompt = f"""
    For the recipe enclosed in the triple back ticks output a python list
    containing the name of the recipe(consist of letters and spaces only. No special characters) as the first entry and a dictionary as the second entry with the following conditions:
    1. The keys should be item names and values the quantity.
    2. If there is no quantity listed give the  default value as 1.
    3. The value should just be a number. There should be no unit of measurement.
    4. Weight should be measured in kg. For example, 500 gram should be made to be 0.5 .
    5. Volume should always be in liters. For example, 150 ml should be made to .15.
    ```{text}```
    """
    response = get_completion(prompt)
    name_end=0
    recipe_start=0
    for i in range(2, len(response)):
        if response[i]=="'":
            name_end=i
        if response[i]=="{":
            recipe_start=i
            break
    json_acceptable_string = response[recipe_start:-1].replace("'", "\"")
    recipe = json.loads(json_acceptable_string)
    recipe_name= response[2:name_end]
    return recipe_name,recipe

## For debugging purposes

In [14]:
prompt = f"""
    For the recipe enclosed in the triple back ticks output a python list
    containing the name of the recipe(consist of letters and spaces only. No special characters) as the first entry and a dictionary as the second entry 
    where the keys should be item names and values the quantity.
    To calulate the quantity for each item follow the following algorithm:
    Step 1. If weight is in grams convert it to kg. For example, 500 gram should be made to be 0.5 kg.
    Step 2. If quantity is in ml convert to liters. For example, 150 ml should be made to .15 l.
    Step 3. Cups should be converted to liters. 1 cup tomatoes should be listed as "tomatoes": 0.25.
    Step 4. Set the quantity to be just a number. There should be no unit of measurement.
    Step 5. The value should be absolute. If there is a range indicated then pick the lower value of the range.
    Step 6. If there is no quantity listed give the  default value as 1.
    ```{raw_recipe}```
    """
response = get_completion(prompt)

In [15]:
response

"['Paneer Butter Masala', {'Oil': 1, 'Paneer cubes': 0.5, 'Salt': 0.5, 'Red chilli powder': 2, 'Butter': 2, 'Black cardamom': 1, 'Cinnamon stick': 1, 'Green cardamom': 5, 'Ginger': 1, 'Garlic': 1, 'Onion, chopped': 0.5, 'Tomato, chopped': 1, 'Cashews': 7, 'Green chilli, chopped': 0.25, 'Coriander powder': 0.5, 'Cumin powder': 0.5, 'Garam masala': 0.25, 'Black pepper': 0.25, 'Kasoori methi': 0.25, 'Honey': 0.5, 'Milk': 0.75, 'Cream': 1, 'Cardamom powder': 0.5}]"

In [48]:
name_end=0
recipe_start=0
for i in range(2, len(response)):
    if response[i]=="'":
        name_end=i
    if response[i]=="{":
        recipe_start=i
        break
json_acceptable_string = response[recipe_start:-1].replace("'", "\"")
recipe = json.loads(json_acceptable_string)
recipe_name= response[2:name_end-1]

In [49]:
print(recipe,recipe_name)

{'Oil': 0.5, 'Paneer cubes': 1, 'Salt': 0.5, 'Red chilli powder': 2, 'Butter': 2, 'Black cardamom': 1, 'Cinnamon stick': 1, 'Green cardamom': 5, 'Ginger': 1, 'Garlic': 1, 'Onion, chopped': 0.5, 'Tomato, chopped': 1, 'Cashews': 7, 'Green chilli, chopped': 0.25, 'Coriander powder': 0.5, 'Cumin powder': 0.5, 'Garam masala': 0.25, 'Black pepper': 0.25, 'Kasoori methi': 0.5, 'Honey': 0.5, 'Milk': 0.75, 'Cream': 1, 'Cardamom powder': 0.5} Paneer Butter Masal


# Back to adding new recipe

In [33]:
recipe_name,recipe= recipe_json(raw_recipe)

JSONDecodeError: Expecting ',' delimiter: line 1 column 228 (char 227)

In [15]:
print(recipe_name,recipe)

Greek stuffed vegetables - Gemista {'baby potatoes': 0.5, 'olive oil': 2, 'salt': 1, 'pepper': 1, 'onions': 3, 'tomatoes': 6, 'green bell peppers': 3, 'round grain rice': 0.35, 'water': 0.4, 'vegetable bouillon cube': 1, 'garlic': 2, 'tomato paste': 1, 'zucchini': 1, 'carrot': 1, 'granulated sugar': 1, 'parsley': 1, 'mint': 1, 'dill': 0.33}


In [17]:
recipe_name= recipe_name.lower().replace(' ', '_')
json_filename = f"{recipe_name}.json"
with open(json_filename, "w") as json_file:
    json.dump(recipe, json_file, indent=4)

print(f"JSON file '{json_filename}' created successfully!")

JSON file 'greek_stuffed_vegetables_-_gemista.json' created successfully!


## Creating shopping list

In [22]:
# Items already in the pantry
pantry_list=["olive oil", "water" , "salt" ,"pepper", "honey", "oregano", "rosemary"]
recipe_list=["gyro", "yemista"]


In [23]:
def parse_input(user_input: str, match_list: list):  # Yiqian
    """Takes a string and finds the correct engagement by parsing it.
    For example, if I input avdivka it should output ['Avdiivka'].
    Args: String engagement_name, list engagement lists.
    Output: a list of matched engagement name."""

    # Initialize cutoff ratio for difflib matching
    cf = 0.0

    # Get a list of close matches to the user input from the match list
    match = difflib.get_close_matches(user_input, match_list, n = 5, cutoff = cf)

    # If more than one match is found, incrementally increase the cutoff ratio to narrow down the matches
    while len(match) > 1:
        cf = cf + 0.01
        match = difflib.get_close_matches(user_input, match_list, n = 5, cutoff = cf)

    # Return the list of matched engagement names
    return match

In [24]:
def correct_list(recipe_list,full_recipe_list):
    corrected_recipe_list=[]
    for item in recipe_list:
        corrected_recipe_list.append(parse_input(item, full_recipe_list)[0])
    print(f"The corrected recipe list is {corrected_recipe_list}")
    return corrected_recipe_list
    

In [25]:

full_recipe_list=["gyros_burger", "greek_stuffed_vegetables_-_gemista", "greek_beef_stew_with_hilopites_pasta"]
corrected_recipe_list=correct_list(recipe_list, full_recipe_list)


The corrected recipe list is ['gyros_burger', 'greek_stuffed_vegetables_-_gemista']


In [27]:
def create_shopping_list(recipe_list,pantry_list):
    shopping_list={}
    for recipe in recipe_list:
        # Opening JSON file
        f = open(f'{recipe}.json')

        # returns JSON object as
        # a dictionary
        data = json.load(f)
        for keys in data:
            if keys in shopping_list.keys():
                shopping_list[keys]=shopping_list[keys]+data[keys]
            else:
                shopping_list[keys]=data[keys]

        # Closing file
        f.close()
    for item in pantry_list:
        for shopping_item in list(shopping_list):
            if item in shopping_item:
                del shopping_list[shopping_item]
    return shopping_list

In [29]:
shopping_list=create_shopping_list(corrected_recipe_list, pantry_list)

olive oil overlap
salt overlap
pepper overlap
granulated sugar overlap


In [30]:
print(shopping_list)

{'cucumber': 1, 'vinegar': 5, 'granulated sugar': 1.15, 'tenderloin': 0.65, 'onion': 1, 'garlic cloves': 2, 'cheddar slices': 6, 'burger buns': 6, 'strained yogurt': 3, 'tahini': 1, 'baby potatoes': 0.5, 'onions': 3, 'tomatoes': 6, 'round grain rice': 0.35, 'vegetable bouillon cube': 1, 'garlic': 2, 'tomato paste': 1, 'zucchini': 1, 'carrot': 1, 'parsley': 1, 'mint': 1, 'dill': 0.33}
